### 人类活动识别——条件扩散

In [ ]:
# 加载类别信息

from utils.ConfigUtils import get_classes

classes=list(get_classes("./data/RFID_multi_628/data.yml").values())
num_classes = len(classes)

print(classes)

In [ ]:
# 数据预处理
from torchvision import transforms
from model.Normalization import RobustNorm
from model.Augmentation import NoiseAug

transform=transforms.Compose([
    RobustNorm(-68.0, 68.0),
    # NoiseAug(0.2,std=(0.001))
])
# transform=None

In [ ]:
# 加载数据集
from model.RFID_Dataset import RFID_Dataset

train_dir = "./data/RFID_multi_628/dataset/train"
eval_dir = "./data/RFID_multi_628/dataset/eval"

train_dataset = RFID_Dataset(
    train_dir,
    T=32,
    step=1,
    num_channels=3,
    transform=transform,
)

eval_dataset = RFID_Dataset(
    eval_dir,
    T=32,
    step=1,
    num_channels=3,
    transform=transform,
)

print(f"训练集的数据个数: {len(train_dataset)}")
print(f"验证集的数据个数: {len(eval_dataset)}")

In [ ]:
# 模型组网

from model.base.UNet import UNet
# from model.v1.UNet import UNet
# from model.v2.UNet import UNet
# from model.v3.UNet import UNet
# from model.v4.UNet import UNet

from model.BetaScheduler import LinearBetaScheduler
from model.CD_Model import CD_Model
from model.ModelWorker.CDModelWorker import CDModelWorker
import torch
from torchkeras import summary

input_shape = (3, 32, 12)

model = CD_Model(
    UNet(
        input_shape=input_shape,
        init_features=64,
        embed_dim=128,
        num_heads=1,
        num_groups=16,
    ),
    LinearBetaScheduler(timesteps=1000,beta_end=0.02),
    num_classes=6,
    embed_dim=128,
    enable_guidance=True,
)

model_worker = CDModelWorker(model)

print(f"{input_shape=}")

time = torch.tensor([0], dtype=torch.long)
condition = torch.tensor([0], dtype=torch.long)
model_info = summary(model, input_shape=input_shape, time=time, condition=condition)

In [ ]:
# 模型准备
from torch.utils.data import DataLoader
from torch import nn
from torch import optim
from model.Loss import *

train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True,num_workers=4)
eval_loader = DataLoader(eval_dataset, batch_size=256,num_workers=4)

loss = nn.MSELoss()
# loss = MinSNRLoss()
# loss=SigmoidLoss()
optimizer = optim.AdamW(model.parameters(), lr=0.0001, weight_decay=0.0001)
# scheduler=optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.98)
scheduler=optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=100)

In [ ]:
model.scheduler.timesteps=1000

In [ ]:
# 模型训练

model_worker.train(
    criterion=loss,
    optimizer=optimizer,
    train_loader=train_loader,
    # eval_loader=eval_loader,
    epochs=200, 
    scheduler=scheduler,
    cond_dropout_rate=0.1, 
    enable_board=False,
    verbose=1
)

In [ ]:
# 模型评估

model_worker.evaluate(
    eval_loader=train_loader,
    criterion=nn.MSELoss(),
    step=10,
    verbose=2
)

In [ ]:
# 模型评估-时间步序列
import torch
from torch import nn

sequence=torch.linspace(0, 1000,10+1,dtype=torch.long).tolist()[1:]
print(sequence)

eval_loss=nn.MSELoss()
# eval_loss=ConstantLoss()
loss_group=model_worker.evaluate_sequence(
    eval_loader=train_loader,
    criterion=eval_loss,
    time=sequence,
    verbose=1
)
for item in loss_group.items():
    print(item)

In [ ]:
from utils.JsonUtils import *

info={
    "epoch":epoch,
    "loss":loss_group
}
append_data(info, "./output/HAR_CD/base/loss.json")

In [ ]:
from utils.JsonUtils import load_data
from utils.DataUtils.Visualization import plot_curves

data=load_data("./output/HAR_CD/base/loss.json")
loss_data = {}
for item in data:
    loss_data[item["epoch"]] = item["loss"]

plot_curves(
    loss_data,
)


In [ ]:
# 保存模型
model_worker.save(f'./output/HAR_CD/base_MSE/weights/HAR_CD_1600.pth')

In [ ]:
# 加载模型
model_worker.load("./output/HAR_CD/base_MSE/weights/HAR_CD_1600.pth")

In [ ]:
epoch=200

In [ ]:
# DDPM采样
from model.RFID_Dataset import save_samples

for batch in range(1):
    print(f"{batch=}")
    for i in range(num_classes):
        # 生成数据
        condition = i
        datas = model_worker.generate_sample_batch(
            100,
            condition,
            guidance_scale=2,
        )

        # 保存数据
        save_samples(
            datas, 
            output_dir=f"./output/base_1600/{condition}",
            merge=True,
        )
    print("=" * 20)

In [ ]:
from torchvision import transforms
from model.RFID_Dataset import build_class_datasets
from model.Normalization import RobustNorm

train_transform=transforms.Compose([
    RobustNorm(-68.0, 68.0),
    transforms.Resize(size=(299,299)),
    transforms.Lambda(lambda x: x.repeat(3, 1, 1)),
])

# 训练集
train_datasets=build_class_datasets(
    "./data/RFID_multi_628/dataset/train",
    T=32,
    step=1,
    num_channels=3,
    transforms=train_transform,
)
for label,dataset in train_datasets.items():
    print(f"label {label} 数据个数为: {len(dataset)}")

In [ ]:
from torchvision import transforms
from model.RFID_Dataset import build_class_datasets
from model.Normalization import RobustNorm

eval_transform=transforms.Compose([
    RobustNorm(-1.0, 1.0),
    transforms.Resize(size=(299,299)),
    transforms.Lambda(lambda x: x.repeat(3, 1, 1)),
])

# 测试集
test_datasets=build_class_datasets(
    f"./output/base_1600",
    T=32,
    step=32,
    num_channels=3,
    transforms=eval_transform,
)
for label,dataset in test_datasets.items():
    print(f"label {label} 数据个数为: {len(dataset)}")

In [ ]:
from utils.JsonUtils import *

info={
    "epoch":epoch,
    "fid":fid_group
}
append_data(info, "./output/HAR_CD/base/fid.json")

In [ ]:
from utils.JsonUtils import load_data
from utils.DataUtils.Visualization import plot_curves

data=load_data("./output/HAR_CD/base/fid.json")
loss_data = {}
for item in data:
    loss_data[item["epoch"]] = item["fid"]

plot_curves(
    loss_data,
)
